In [1]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np
import os
from PIL import Image
import torch.nn as nn

# CNN 모델 정의
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv1(x), 2))
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x  # nn.functional.log_softmax 사용 제거


In [2]:

# 모델 불러오기
model = SimpleCNN()
model.load_state_dict(torch.load('C:/Users/koo/my_workspace/privacy_term_project/mnist_cnn.pth'))
model.eval()
model.cuda()  # GPU 사용 시


SimpleCNN(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [3]:

# 노이즈 마스크 생성 함수
def add_noise_to_image(image, num_pixels, intensity=0.5):
    image = image.clone()
    for _ in range(num_pixels):
        x, y = np.random.randint(0, image.size(1)), np.random.randint(0, image.size(2))
        noise = torch.randn(image.size(0)) * intensity
        image[:, x, y] += noise
    return torch.clamp(image, 0, 1)


In [4]:

# 데이터 로딩 및 폴더 생성
test_dir = 'C:/Users/koo/my_workspace/privacy_term_project/mnist_btm/testing'
output_dir = 'C:/Users/koo/my_workspace/privacy_term_project/random_pertubation_image'

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
test_dataset = ImageFolder(root=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)


In [5]:

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    for i in range(10):
        os.makedirs(os.path.join(output_dir, str(i)))


In [6]:

# 이미지 생성 및 모델 테스트
success_counts = {str(i): 0 for i in range(10)}
total_counts = {str(i): 0 for i in range(10)}


In [7]:

# 노이즈 추가 및 모델 예측 부분

for label in range(10):
    label_dir = f'{output_dir}/{label}'
    if not os.path.exists(label_dir):
        os.makedirs(label_dir)
        
    for i, (data, target) in enumerate(test_loader):
        if target.item() != label or total_counts[str(label)] >= 1000:
            continue

        noisy_image = add_noise_to_image(data[0], num_pixels=5, intensity=0.5)
        noisy_image = noisy_image.unsqueeze(0).cuda()
        prediction = model(noisy_image)
        predicted_label = torch.max(prediction, 1)[1].item()

        if predicted_label != target.item():
            success_counts[str(label)] += 1
            noisy_img = transforms.ToPILImage()(noisy_image.squeeze(0).cpu())  # 배치 차원 제거 및 CPU로 이동
            noisy_img.save(f'{output_dir}/{label}/{label}_noisy_{total_counts[str(label)]}.bmp')

        total_counts[str(label)] += 1


In [15]:
# 성공률 출력
for label in range(10):
    success_rate = success_counts[str(label)] / 1000 * 100
    print(f'Label {label}: Success Rate = {success_rate}%')
#     print("  success_counts:" + str(success_counts[str(label)]))
#     print("  total_counts:" + str(total_counts[str(label)]))
    
# Label 0: Success Rate = 1.9%
# Label 1: Success Rate = 2.9000000000000004%
# Label 2: Success Rate = 6.5%
# Label 3: Success Rate = 1.6%
# Label 4: Success Rate = 1.9%
# Label 5: Success Rate = 11.700000000000001%
# Label 6: Success Rate = 8.799999999999999%
# Label 7: Success Rate = 4.1000000000000005%
# Label 8: Success Rate = 0.4%
# Label 9: Success Rate = 3.9%

Label 0: Success Rate = 1.9%
Label 1: Success Rate = 2.9000000000000004%
Label 2: Success Rate = 6.5%
Label 3: Success Rate = 1.6%
Label 4: Success Rate = 1.9%
Label 5: Success Rate = 11.700000000000001%
Label 6: Success Rate = 8.799999999999999%
Label 7: Success Rate = 4.1000000000000005%
Label 8: Success Rate = 0.4%
Label 9: Success Rate = 3.9%


In [14]:
s_sum = 0
t_sum = 0
for label in range(10):
    s_sum += success_counts[str(label)] 
    t_sum += total_counts[str(label)] 
print(str(s_sum/t_sum*100) + "%")
#4.37 %

4.465563049254037%
